In [1]:
import os
import sys
import pickle
import pandas as pd
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# Obtener la ruta del directorio base (nivel superior del proyecto)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Agregar SRC_PATH a sys.path si no está presente
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

In [3]:
with open(r'..\models\Best_Model_RF.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)


In [4]:
# Importar librerías necesarias
# Cargar datasets
df_tr = pd.read_csv(r"..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_trtest.csv")[[
    'nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'var_rpta_alt'
]]
df_hist_scores = pd.read_csv(r"..\data\raw\prueba_op_probabilidad_oblig_base_hist_enmascarado_completa.csv")
df_clientes = pd.read_csv(r"..\data\raw\prueba_op_master_customer_data_enmascarado_completa.csv")
df_pagos = pd.read_csv(r"..\data\raw\prueba_op_maestra_cuotas_pagos_mes_hist_enmascarado_completa.csv")

# Ajustar formato de 'fecha_corte' en df_pagos y df_clientes
df_pagos['fecha_corte'] = df_pagos['fecha_corte'].astype(str).str[:6].astype(int)
df_clientes['fecha_corte'] = df_clientes['year'].astype(str) + df_clientes['month'].astype(str).str.zfill(2)

# Ordenar y eliminar duplicados en df_hist_scores
df_hist_scores.sort_values(by=['fecha_corte'], ascending=False, inplace=True)
df_hist_scores.drop_duplicates(subset=['nit_enmascarado'], keep='first', inplace=True)

# Ordenar y eliminar duplicados en df_clientes
df_clientes.sort_values(by=['fecha_corte'], ascending=False, inplace=True)
df_clientes.drop_duplicates(subset=['nit_enmascarado'], keep='first', inplace=True)

# Ordenar y eliminar duplicados en df_pagos
df_pagos.sort_values(by=['fecha_corte'], ascending=False, inplace=True)
df_pagos.drop_duplicates(subset=['nit_enmascarado'], keep='first', inplace=True)

df_oot = pd.read_csv(r'..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv')

df_test = pd.merge(
    df_oot, df_clientes, 
    on=['nit_enmascarado'], 
    how='left'
)
df_test = pd.merge(
    df_test, df_pagos, 
    on=['nit_enmascarado'], 
    how='left'
)

df_test = pd.merge(
    df_test, df_hist_scores, 
    on=['nit_enmascarado'], 
    how='left'
)

df_test['id'] = df_test['nit_enmascarado'].astype(str) + '#' + df_test['num_oblig_orig_enmascarado'].astype(str) + '#' + df_test['num_oblig_enmascarado'].astype(str)
df_test.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [6]:
features = pd.read_csv(r'..\data\procesed\features.csv')

In [7]:
from data_engineer import FeatureSelector

limpieza = FeatureSelector(df_test, features)
df = limpieza.fit_transform()

Limpiando variables numéricas...


d:\Desktop\juan_guzman_prueba\env\lib\site-packages\pandas\core\nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean


Modo inferencia: No se encontró 'var_rpta_alt'
Detectando tipos de variables...
Preprocesando variables...
Transformando variables...
Reconstruyendo nombres de columnas...
Seleccionando variables...
Ajustando columnas faltantes...
Ordenando columnas...
Finalizando DataFrame...


In [8]:
# 2. Seleccionar columnas que usó el modelo
columnas_modelo = modelo.feature_names_in_
X = df[columnas_modelo]

# 3. Predecir
predicciones = modelo.predict(X)

# 4. Crear DataFrame final
df_submission = pd.DataFrame({
    'id': df['id'],
    'var_rpta_alt': predicciones,
    'var_rpta_alt_prob': modelo.predict_proba(X)[:, 1]
})

# 5. Guardar en CSV con el formato solicitado
df_submission.to_csv('submission.csv', index=False)


In [9]:
df_submission

,id,var_rpta_alt,var_rpta_alt_prob
0,257335#444821#635511.0,0.0,0.312880
1,59584#350400#730364.0,0.0,0.345997
2,397604#973821#106513.0,0.0,0.322988
3,368086#382995#696856.0,0.0,0.498335
4,255009#434238#953589.0,0.0,0.320728
...,...,...,...
112542,306471#5783#883781.0,0.0,0.191075
112543,32742#11429#1078300.0,1.0,0.509126
112544,316499#14503#536986.0,0.0,0.262533
112546,483193#13732#310671.0,0.0,0.223419
